In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
import string
def delete_punctuation(x):
    return ''.join([a for a in x if a not in string.punctuation])

In [3]:
# Based on Shengy's code:
# http://stackoverflow.com/questions/18840640/python-2-7-find-and-replace-from-text-file-using-dictionary-to-new-text-file

def replace_all(text, mydict):
    for gb, us in mydict.items():
        text = text.replace(us, gb)
    return text

In [4]:
mydict = {'behaviour': 'behavior',
         'favourable': 'favorable',
         'labour': 'labor',
         'organisation': 'organization'}

In [5]:
# Based on DataCamp article:
# http://datacamp.com/community/tutorials/stemming-lemmatization-python

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
# from nltk.stem import PorterStemmer
lemmatizer = WordNetLemmatizer()
# porter = PorterStemmer()

def stemText(text):
    token_words = word_tokenize(text)
    stem_text = list()
    for word in token_words:
        stem_text.append(lemmatizer.lemmatize(word))
        # stem_text.append(porter.stem(word))
        stem_text.append(' ')
    return ''.join(stem_text)

In [6]:
# capital = os.listdir(os.getcwd() + '/Classes')
# for i in range(len(capital)):
    # capital[i] = capital[i].replace('.txt', '')
capital = ['structural', 'human', 'relational', 'social', 'natural']

In [7]:
for item in capital:
    file = open('Classes/' + item + '.txt')
    print(item.upper())
    for line in file.readlines():
        print(line.replace('\n', ''))
    if not item == capital[len(capital) - 1]:
        print()

STRUCTURAL
Accreditation
Certificate
Corporate culture
Customer support function
Distribution network
Financial dealings
Innovation
Intellectual property
Knowledge-based infrastructure
Management philosophy
Networking
Organisation flexibility
Organisation learning
Organisation structure
Overall capability
Overall infrastructure
Process
Quality management and improvement
Research and development
Technology

HUMAN
Employee age
Employee attitude
Employee behaviour
Employee capability
Employee commitment
Employee development
Employee diversity
Employee education
Employee equality
Employee flexibility
Employee involvement with community
Employee motivation
Employee productivity
Employee relationship
Employee teamwork
Employee training
Employee work-related competence
Employee work-related knowledge
Entrepreneurial spirit
Know-how
Number of employees
Skill
Vocational qualification

RELATIONAL
Acquaintance with community
Acquaintance with government
Acquaintance with suppliers
Basic marketing

In [8]:
keywords = list()
for item in capital:
    file = open('Classes/' + item + '.txt')
    text = delete_punctuation(file.read().replace('\n', ' zzz ').replace('  ', ' ').replace('  ', ' ').lower())
    text = stemText(replace_all(text, mydict))
    newlist = text.split('zzz')
    for i in range(len(newlist)):
        newlist[i] = newlist[i].strip()
    keywords.append([item, newlist])
# keywords

In [9]:
for item in keywords:
    print(item[0] + ': ' + str(len(item[1])))

structural: 20
human: 23
relational: 43
social: 33
natural: 35


In [10]:
files = sorted(os.listdir(os.getcwd() + '/Files'))

In [11]:
df = list()
for filename in files:
    file = open('Files/' + filename)
    text = delete_punctuation(file.read().replace('\n', ' ').replace('  ', ' ').replace('  ', ' ').lower())
    text = stemText(replace_all(text, mydict))
    df.append([])
    for item in keywords:
        count = 0
        for word in item[1]:
            if not text.find(word) == -1:
                count += 1
        df[len(df) - 1].append(count)

In [12]:
df = np.array(df)
overall = df.sum(axis=1)

files = pd.DataFrame(files)
df = pd.DataFrame(df)
overall = pd.DataFrame(overall)

df = pd.concat([files, df, overall], axis=1, ignore_index=True)
df.columns = ['name'] + capital + ['overall']
df.index = df['name']
del df['name']

dff = pd.read_csv('performance.csv', sep=';')
dff.index = df.index
df = pd.concat([df, dff], axis=1, ignore_index=True)
df.columns = capital + ['overall'] + list(dff.columns)

In [13]:
df.head()

,structural,human,relational,social,natural,overall,current_assets,non-current_assets,current_liab,non-current_liab,equity,revenue,revenue_prev,EBIT,EBIT_prev,net_operat_CF,units_EUR
name,,,,,,,,,,,,,,,,,
Adidas_IR_2014.txt,5,2,6,10,3,26,7497.0,5846.0,5364.0,2332.0,5648.0,16915.0,14534.0,1059.0,883.0,1090.0,1000000
Adidas_IR_2015.txt,5,2,6,11,7,31,8886.0,6290.0,6765.0,1957.0,6455.0,19291.0,16915.0,1491.0,1059.0,1348.0,1000000
Adidas_IR_2016.txt,3,3,4,8,10,28,8645.0,5877.0,6291.0,1796.0,6435.0,21218.0,18483.0,2070.0,1582.0,1648.0,1000000
Audi_IR_2014.txt,8,3,10,17,19,57,30800.0,25963.0,21554.0,13431.0,21779.0,58420.0,53787.0,4836.0,5150.0,7203.0,1000000
Audi_IR_2015.txt,3,1,3,3,7,17,32490.0,28599.0,20789.0,14980.0,25321.0,59317.0,58240.0,3052.0,4836.0,7517.0,1000000


In [14]:
df.to_excel('export.xlsx')